In [ ]:
import os, io, shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, log_loss, classification_report, confusion_matrix

In [ ]:
def load_colored_mnist(npz_path):
    """
    colored_mnist.npz를 로드하고 pandas DataFrame(train_df, test_df) 형태로 반환.
    로컬 환경에서도 I/O 오류 없이 안정적으로 동작.
    """
    if not os.path.exists(npz_path):
        raise FileNotFoundError(f"❌ 파일이 없습니다: {npz_path}")

    # 안전한 로드 (BytesIO 사용)
    with open(npz_path, "rb") as f:
        file_bytes = io.BytesIO(f.read())

    data = np.load(file_bytes, allow_pickle=True)
    print(f"[OK] Loaded npz from: {npz_path}")
    print(f"Available keys: {list(data.keys())}")

    # 표준화본(X_train_std 등)이 있으면 사용, 없으면 원본 사용
    X_train = data["X_train_std"] if "X_train_std" in data else data["X_train"]
    X_test  = data["X_test_std"]  if "X_test_std"  in data else data["X_test"]
    y_train = data["y_digit_train"]
    y_test  = data["y_digit_test"]

    # (N, H, W, C) → (N, -1)
    X_train_flat = X_train.reshape(len(X_train), -1)
    X_test_flat  = X_test.reshape(len(X_test), -1)

    train_df = pd.DataFrame(X_train_flat)
    train_df["label"] = y_train
    test_df = pd.DataFrame(X_test_flat)
    test_df["label"] = y_test

    print(f"[INFO] Train shape: {train_df.shape}, Test shape: {test_df.shape}")
    return train_df, test_df


In [ ]:
def train_logistic_regression(train_df, test_df, C=1.0, max_iter=1000, seed=42):
    """
    다항 로지스틱 회귀 학습 및 평가 (CPU 전용)
    """
    X_train = train_df.drop(columns=["label"]).values
    y_train = train_df["label"].values
    X_test  = test_df.drop(columns=["label"]).values
    y_test  = test_df["label"].values

    # 표준화
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test  = scaler.transform(X_test)

    # 모델 정의 (FutureWarning 방지 → multi_class 옵션 제거)
    model = LogisticRegression(
        solver="lbfgs",
        C=C,
        max_iter=max_iter,
        random_state=seed
    )

    print("[INFO] Training Logistic Regression (CPU)...")
    model.fit(X_train, y_train)

    # 예측
    y_prob = model.predict_proba(X_test)
    y_pred = np.argmax(y_prob, axis=1)

    # 평가
    acc = accuracy_score(y_test, y_pred)
    ce = log_loss(y_test, y_prob)

    print(f"\n✅ Accuracy: {acc:.4f}")
    print(f"✅ CrossEntropy (log_loss): {ce:.6f}\n")
    print("[Classification Report]")
    print(classification_report(y_test, y_pred, digits=4))

    # 혼동행렬 시각화
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6,5), dpi=140)
    plt.imshow(cm, cmap="Blues")
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.colorbar()
    plt.tight_layout()
    plt.show()

    return model, scaler

In [ ]:
def main():
    
    # ⚠️ 여기를 네 로컬 경로로 바꿔줘!
    npz_path = "C:/src/colored-mnist-classification/data/processed/colored_mnist.npz"
    out_dir = "./outputs"
    Path(out_dir).mkdir(parents=True, exist_ok=True)

    # 데이터 로드
    train_df, test_df = load_colored_mnist(npz_path)

    # 모델 학습
    model, scaler = train_logistic_regression(
        train_df=train_df,
        test_df=test_df,
        C=1.0,
        max_iter=1000,
        seed=42
    )

    # 모델 저장
    model_path = os.path.join(out_dir, "logreg_model.joblib")
    joblib.dump({"model": model, "scaler": scaler}, model_path)
    print(f"[✅] Model saved at: {model_path}")
    print("\n🎯 All done!")

In [ ]:
main()